In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [2]:
#Libraries
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
train_df = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
test_df = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")

In [4]:
train_df.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [5]:
train_prompts.head()

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [6]:
train_df['prompt_id'].value_counts()

prompt_id
0    708
1    670
Name: count, dtype: int64

In [7]:
train_df['generated'].value_counts()

generated
0    1375
1       3
Name: count, dtype: int64

In [8]:
train_prompts['prompt_id'].value_counts()

prompt_id
0    1
1    1
Name: count, dtype: int64

Since in the training data there are only few LLM generated texts , getting the prompt used and generating few more LLM samples can help the model to perfrom better.

In [9]:
def prompt_id_to_prompt(prompt_id):
    if prompt_id==0:
        return "Car-free cities"
    else:
        return "Does the electoral college work?"

In [10]:
train_df['prompt_name'] = train_df['prompt_id'].apply(prompt_id_to_prompt)
train_df.head()

,id,prompt_id,text,generated,prompt_name
0,0059830c,0,Cars. Cars have been around since they became ...,0,Car-free cities
1,005db917,0,Transportation is a large necessity in most co...,0,Car-free cities
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0,Car-free cities
3,00940276,0,How often do you ride in a car? Do you drive a...,0,Car-free cities
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0,Car-free cities


In [11]:
#ANALYSING size of texts
train_df['length'] = train_df['text'].str.len()


In [12]:
print(f"Max Length : {train_df['length'].max()}")
print(f"Min Length : {train_df['length'].min()}")

Max Length : 8436
Min Length : 1356


Manually generating few chat GPT chatGPT based essays

In [13]:
#Manual CHATGPT generate based ob "Car free cites"
#20 essays
chatgpt_essays = [
    """Car-free cities represent a paradigm shift in urban planning, focusing on sustainability and community well-being. By discouraging private vehicle use and promoting alternative transportation modes, these cities aim to reduce environmental impact and enhance overall quality of life.""",

    """Traffic congestion is a pervasive issue in urban areas, leading to wasted time, increased pollution, and heightened stress levels. Car-free city initiatives address these challenges by reimagining transportation infrastructure and prioritizing pedestrian-friendly spaces.""",

    """The success of car-free cities relies on robust public transportation systems. Efficient buses, trams, and subways provide viable alternatives to private cars, ensuring that residents can navigate the city seamlessly without contributing to traffic congestion.""",

    """Pedestrian zones, where cars are entirely restricted, create vibrant urban spaces. These areas become hubs for cultural activities, social interactions, and community events, fostering a sense of belonging among residents.""",

    """Sustainability is a key driver behind the push for car-free cities. By reducing reliance on fossil fuel-powered vehicles, these cities contribute to lower carbon emissions, mitigating the impact of climate change and improving air quality.""",

    """Green transportation options, such as electric bicycles and shared mobility services, play a pivotal role in the transition to car-free cities. These alternatives provide flexibility and convenience without compromising environmental goals.""",

    """Car-free zones are catalysts for economic revitalization. Local businesses flourish as people are more inclined to explore shops and cafes when freed from the constraints of heavy traffic.""",

    """Implementing car-free initiatives requires comprehensive urban planning. Cities must consider infrastructure changes, alternative transportation options, and public awareness campaigns to ensure a smooth transition and widespread support.""",

    """While the benefits of car-free cities are clear, challenges such as resistance to change and the need for extensive public transportation infrastructure must be addressed to make these initiatives successful.""",

    """Car-free neighborhoods within larger cities serve as experiments in sustainable living. These smaller-scale initiatives allow for testing and refining before implementing broader car-free policies across entire urban areas.""",

    """Car-free cities align with the principles of the New Urbanism movement, emphasizing walkability, mixed land use, and community engagement. These principles contribute to a more connected and socially vibrant urban landscape.""",

    """Historically, some cities have successfully implemented car-free zones on specific days or during certain events. Expanding these initiatives to a full-time, city-wide scale requires careful planning and consideration of diverse urban needs.""",

    """Public support is crucial for the success of car-free city initiatives. Engaging residents through community forums, education campaigns, and involving them in the decision-making process fosters a sense of ownership and commitment to the transformation.""",

    """Urban planners must address accessibility concerns when implementing car-free initiatives. Ensuring that public transportation is inclusive and accommodates individuals with diverse mobility needs is paramount for success.""",

    """Car-free cities are not about restricting mobility but rather redefining it. By encouraging sustainable modes of transportation, these cities offer residents a choice that aligns with both personal and collective well-being.""",

    """Economic analyses of car-free city projects reveal potential cost savings in healthcare, reduced traffic-related expenditures, and increased economic activity in pedestrian-oriented commercial areas.""",

    """Car-free cities contribute to a shift in mindset regarding transportation. Prioritizing active modes of transport not only reduces the environmental impact but also promotes healthier lifestyles and physical well-being.""",

    """Technology plays a role in the success of car-free cities. Smart traffic management systems, ride-sharing apps, and sustainable mobility solutions contribute to the efficiency and effectiveness of alternative transportation.""",

    """Car-free initiatives are not one-size-fits-all solutions. Cities must tailor their approaches to accommodate unique geographical, cultural, and socioeconomic factors, ensuring that the transition aligns with the specific needs of their residents.""",

    """As more cities explore the possibility of becoming car-free, the sharing of best practices and lessons learned becomes crucial. Collaboration among cities can accelerate the adoption of sustainable urban living and create a global network of car-free advocates.""",
]
len(chatgpt_essays)


20

In [14]:
chatgpt_esssays_2 = essays = [
    """Car-free cities are visionary urban spaces that prioritize the well-being of both the environment and inhabitants. By discouraging the use of private vehicles, these cities aim to reduce pollution, alleviate traffic congestion, and create more livable communities.""",

    """The call for car-free cities is not merely an environmental initiative but a holistic approach to urban planning. It involves reimagining transportation systems, redesigning public spaces, and fostering a cultural shift towards sustainable living.""",

    """One of the challenges in transitioning to a car-free city is the need for robust public transportation infrastructure. Efficient buses, trams, and metro systems are crucial to ensuring that residents can move seamlessly without relying on private cars.""",

    """Car-free zones contribute to economic revitalization by creating pedestrian-friendly areas where local businesses can thrive. The absence of vehicular traffic encourages people to explore shops and cafes, enhancing the vibrancy of commercial districts.""",

    """The success of car-free cities lies in the willingness of residents to embrace alternative modes of transportation. Public awareness campaigns and community engagement play a crucial role in garnering support for such transformative initiatives.""",

    """Car-free initiatives are not a one-size-fits-all solution. Each city must tailor its approach to consider unique geographical, cultural, and socioeconomic factors, ensuring that the transition aligns with the specific needs of its residents.""",

    """While the benefits of car-free cities are evident, implementation requires careful consideration of accessibility. Ensuring that public transportation is inclusive and accommodates individuals with diverse mobility needs is paramount for success.""",

    """Car-free neighborhoods within larger cities serve as experimental grounds for sustainable living. These smaller-scale initiatives allow for testing and refining before implementing broader car-free policies across entire urban areas.""",

    """The concept of car-free cities aligns with the principles of the New Urbanism movement, emphasizing walkability, mixed land use, and community engagement. These principles contribute to a more connected and socially vibrant urban landscape.""",

    """Car-free cities are not about restricting mobility but rather redefining it. By encouraging sustainable modes of transportation, these cities offer residents a choice that aligns with both personal and collective well-being.""",

    """Implementing car-free initiatives requires comprehensive urban planning. Cities must consider infrastructure changes, alternative transportation options, and public awareness campaigns to ensure a smooth transition and widespread support.""",

    """Car-free zones are catalysts for economic revitalization. Local businesses flourish as people are more inclined to explore shops and cafes when freed from the constraints of heavy traffic.""",

    """Green transportation options, such as electric bicycles and shared mobility services, play a pivotal role in the transition to car-free cities. These alternatives provide flexibility and convenience without compromising environmental goals.""",

    """Car-free cities represent a paradigm shift in urban planning, focusing on sustainability and community well-being. By discouraging private vehicle use and promoting alternative transportation modes, these cities aim to reduce environmental impact and enhance overall quality of life.""",

    """Traffic congestion is a pervasive issue in urban areas, leading to wasted time, increased pollution, and heightened stress levels. Car-free city initiatives address these challenges by reimagining transportation infrastructure and prioritizing pedestrian-friendly spaces.""",

    """The success of car-free cities relies on robust public transportation systems. Efficient buses, trams, and subways provide viable alternatives to private cars, ensuring that residents can navigate the city seamlessly without contributing to traffic congestion.""",

    """Pedestrian zones, where cars are entirely restricted, create vibrant urban spaces. These areas become hubs for cultural activities, social interactions, and community events, fostering a sense of belonging among residents.""",

    """Sustainability is a key driver behind the push for car-free cities. By reducing reliance on fossil fuel-powered vehicles, these cities contribute to lower carbon emissions, mitigating the impact of climate change and improving air quality.""",

    """Green transportation options, such as electric bicycles and shared mobility services, play a pivotal role in the transition to car-free cities. These alternatives provide flexibility and convenience without compromising environmental goals.""",

    """Car-free zones are catalysts for economic revitalization. Local businesses flourish as people are more inclined to explore shops and cafes when freed from the constraints of heavy traffic.""",

    """Implementing car-free initiatives requires comprehensive urban planning. Cities must consider infrastructure changes, alternative transportation options, and public awareness campaigns to ensure a smooth transition and widespread support.""",

    """While the benefits of car-free cities are clear, challenges such as resistance to change and the need for extensive public transportation infrastructure must be addressed to make these initiatives successful.""",

    """Car-free neighborhoods within larger cities serve as experiments in sustainable living. These smaller-scale initiatives allow for testing and refining before implementing broader car-free policies across entire urban areas.""",

    """Car-free cities align with the principles of the New Urbanism movement, emphasizing walkability, mixed land use, and community engagement. These principles contribute to a more connected and socially vibrant urban landscape.""",

    """Historically, some cities have successfully implemented car-free zones on specific days or during certain events. Expanding these initiatives to a full-time, city-wide scale requires careful planning and consideration of diverse urban needs.""",

    """Public support is crucial for the success of car-free city initiatives. Engaging residents through community forums, education campaigns, and involving them in the decision-making process fosters a sense of ownership and commitment to the transformation.""",

    """Urban planners must address accessibility concerns when implementing car-free initiatives. Ensuring that public transportation is inclusive and accommodates individuals with diverse mobility needs is paramount for success.""",

    """Car-free cities are not about restricting mobility but rather redefining it. By encouraging sustainable modes of transportation, these cities offer residents a choice that aligns with both personal and collective well-being.""",

    """Economic analyses of car-free city projects reveal potential cost savings in healthcare, reduced traffic-related expenditures, and increased economic activity in pedestrian-oriented commercial areas.""",

    """Car-free cities contribute to a shift in mindset regarding transportation. Prioritizing active modes of transport not only reduces the environmental impact but also promotes healthier lifestyles and physical well-being.""",

    """Technology plays a role in the success of car-free cities. Smart traffic management systems, ride-sharing apps, and sustainable mobility solutions contribute to the efficiency and effectiveness of alternative transportation.""",

    """Car-free initiatives are not one-size-fits-all solutions. Cities must tailor their approaches to accommodate unique geographical, cultural, and socioeconomic factors, ensuring that the transition aligns with the specific needs of their residents.""",

    """As more cities explore the possibility of becoming car-free, the sharing of best practices and lessons learned becomes crucial. Collaboration among cities can accelerate the adoption of sustainable urban living and create a global network of car-free advocates.""",
]
len(chatgpt_esssays_2)

33

In [15]:
chatgpt_essays_3 = [
    """Car-free cities embody a progressive vision for urban living, challenging the traditional reliance on private vehicles. The underlying philosophy centers on sustainability, environmental consciousness, and the creation of more inclusive, pedestrian-friendly urban spaces.""",

    """The concept of car-free cities isn't just about reducing traffic congestion; it's a holistic approach to redefining the urban experience. By discouraging private car use, cities aim to improve air quality, reduce noise pollution, and create healthier, more livable environments for their residents.""",

    """Urban planners face a delicate balancing act when transitioning to car-free cities. They must redesign transportation systems, allocate space for alternative modes of transport, and consider the diverse needs of the community. The success of such initiatives hinges on meticulous planning and strategic execution.""",

    """Car-free zones, where vehicular traffic is entirely restricted, hold the promise of transforming urban landscapes. Streets once dominated by cars become vibrant spaces for people to walk, cycle, and engage in community activities, fostering a sense of connectivity and shared public spaces.""",

    """Public transportation emerges as a linchpin in the viability of car-free cities. Cities must invest in efficient and accessible public transit systems to provide residents with reliable alternatives to private vehicles. This necessitates not only expanding existing infrastructure but also introducing innovative solutions for last-mile connectivity.""",

    """The economic implications of car-free initiatives extend beyond reduced traffic congestion. By creating pedestrian-oriented zones and promoting sustainable mobility, cities can attract businesses and stimulate local economies. This economic revitalization becomes a driving force for the success and acceptance of car-free urban living.""",

    """Car-free cities embrace a fundamental shift in mindset, encouraging residents to adopt alternative modes of transportation. Electric bicycles, shared mobility services, and improved public transit become not just choices but lifestyle preferences aligned with environmental consciousness and individual well-being.""",

    """A critical consideration in the implementation of car-free cities is social equity. Urban planners must ensure that the benefits of reduced congestion and improved air quality are distributed equitably across diverse communities, avoiding unintended consequences that may disproportionately affect certain groups.""",

    """While the benefits of car-free cities are numerous, these initiatives face resistance from individuals accustomed to the convenience of private vehicle use. Overcoming this resistance requires effective communication, education, and a clear demonstration of the long-term advantages of a car-free urban lifestyle.""",

    """Car-free neighborhoods within larger cities serve as experimental grounds for sustainable living. These smaller-scale initiatives allow for testing and refining before implementing broader car-free policies across entire urban areas.""",

    """In transitioning to car-free cities, technology plays a pivotal role. Smart city solutions, real-time traffic management systems, and innovative transportation apps contribute to the efficiency and effectiveness of alternative transportation methods, ensuring a seamless experience for residents.""",

    """Car-free initiatives necessitate collaboration between governments, urban planners, and the community. Open dialogues, community engagement, and a participatory approach to decision-making foster a sense of shared responsibility and ownership, crucial for the success of these transformative endeavors.""",

    """The transition to car-free cities is not without its challenges. Cities must grapple with the need for comprehensive infrastructure changes, potential disruptions during the transition period, and the intricate task of reshaping the urban fabric to accommodate alternative transportation options.""",

    """Green transportation modes, such as electric bikes and scooters, contribute to the ecological sustainability of car-free cities. These modes offer efficient and environmentally friendly alternatives, aligning with the broader goals of reducing carbon emissions and mitigating the impacts of climate change.""",

    """Car-free cities can redefine public spaces by transforming roadways into green corridors, parks, and communal areas. This shift enhances the aesthetic appeal of urban environments, fostering a sense of pride among residents and creating spaces for recreational activities and cultural events.""",

    """In embracing car-free cities, cities not only address the immediate challenges of traffic congestion and pollution but also contribute to a global movement toward sustainable urban living. The sharing of best practices and lessons learned becomes vital as cities collaborate to create a network of car-free advocates worldwide.""",

    """Car-free zones represent a departure from the conventional urban landscape. These zones prioritize pedestrians, cyclists, and public transit over private vehicles, reshaping the dynamics of transportation and fostering a more inclusive, healthier urban experience.""",

    """Implementing car-free initiatives requires a multidimensional approach, integrating urban planning, policy changes, and community engagement. A phased implementation, starting with pilot projects and gradually expanding, allows cities to test strategies and adapt to the evolving needs of residents.""",

    """Car-free cities offer a solution to the perennial issue of traffic congestion that plagues urban areas worldwide. By reducing the number of private vehicles on the road, cities can create more efficient transportation systems and improve the overall quality of life for their residents.""",

    """The success of car-free cities hinges on the synergy between urban planning and technological innovation. Smart traffic management systems, real-time data analytics, and the integration of sustainable transportation solutions are integral to creating seamless, efficient urban environments that prioritize pedestrian and cyclist-friendly spaces.""",

    """Car-free initiatives necessitate a paradigm shift in how individuals perceive and engage with urban spaces. By reimagining streets as shared public spaces rather than conduits for vehicular traffic, cities can foster a sense of community, encourage social interactions, and promote a healthier, more active lifestyle for their residents.""",

    """Car-free zones serve as living laboratories for sustainable urban living. These areas allow cities to experiment with innovative solutions, test the viability of alternative transportation methods, and gather valuable insights to inform the broader implementation of car-free policies across the entire urban landscape.""",

    """Urban planners face the challenge of striking a balance between car-free initiatives and maintaining accessibility for all residents. Ensuring that public transportation is inclusive, accommodating individuals with diverse mobility needs, is essential for the success and acceptance of car-free urban living.""",

    """Car-free cities embody a vision of urban living that prioritizes environmental sustainability, reduced carbon emissions, and improved air quality. By promoting alternative modes of transportation and limiting the use of private vehicles, cities can create cleaner, healthier environments for their residents.""",

    """The economic benefits of car-free initiatives extend beyond reduced traffic congestion. These initiatives stimulate local economies by creating pedestrian-friendly zones that attract foot traffic, encourage local businesses, and contribute to a vibrant, economically thriving urban landscape.""",

    """Car-free cities offer an opportunity to rethink the allocation of urban space. By reclaiming streets from vehicular traffic, cities can create green corridors, parks, and recreational spaces that enhance the overall quality of life for residents and contribute to a more sustainable, ecologically conscious urban environment.""",

    """In transitioning to car-free cities, fostering a culture of sustainability and environmental consciousness becomes paramount. Education campaigns, community involvement, and initiatives promoting sustainable living practices contribute to the success and acceptance of car-free urban initiatives.""",

    """Car-free initiatives are not isolated endeavors but part of a broader global movement toward sustainable urban living. Cities worldwide can learn from each other's successes and challenges, creating a collaborative network of car-free advocates working toward a common goal of creating more livable, environmentally friendly urban spaces.""",

    """Car-free cities are not about deprivation but empowerment—empowering residents with sustainable transportation options, creating healthier urban environments, and fostering a sense of community and shared responsibility for the well-being of the city and its inhabitants.""",

    """The shift to car-free cities requires a comprehensive reassessment of transportation infrastructure. Investing in alternative modes of transport, expanding public transit options, and integrating smart city solutions are integral components of creating efficient, sustainable urban environments that prioritize the needs of residents over vehicular traffic.""",

    """Car-free zones can serve as catalysts for social change, encouraging people to reevaluate their relationship with personal vehicles and embrace alternative, more sustainable modes of transportation. This shift in mindset is essential for the long-term success of car-free initiatives and the creation of truly livable urban spaces.""",

    """Car-free cities exemplify a forward-thinking approach to urban planning. By placing pedestrians, cyclists, and public transit at the forefront, these cities aim to reduce traffic congestion, minimize environmental impact, and create inclusive, vibrant urban spaces that prioritize the well-being of residents over vehicular convenience.""",

    """The implementation of car-free initiatives requires strategic collaboration between urban planners, policymakers, and the community. Engaging residents in the decision-making process, addressing concerns, and ensuring transparency are crucial elements for garnering support and fostering a sense of ownership in the transformative journey toward car-free urban living.""",

    """Car-free cities are not just a solution to traffic woes; they represent a commitment to creating healthier, more sustainable urban environments. By promoting active modes of transportation and reducing the reliance on private vehicles, cities can improve air quality, reduce noise pollution, and enhance the overall quality of life for their residents.""",

    """The success of car-free cities depends on the adaptability of urban planning to diverse cultural and geographic contexts. Customizing strategies, considering local needs, and integrating community feedback ensure that car-free initiatives align with the specific characteristics and requirements of each unique urban landscape.""",

    """Car-free initiatives offer a unique opportunity for cities to redefine public spaces and enhance the urban experience. Streets once dominated by traffic become places for socializing, recreation, and cultural events, fostering a stronger sense of community and connection among residents.""",

    """The integration of technology is a linchpin in the success of car-free cities. Smart traffic management systems, real-time data analytics, and innovative transportation solutions contribute to the efficiency and effectiveness of alternative modes of transportation, ensuring a seamless transition to car-free urban living.""",

    """Car-free cities are not just about reducing the number of vehicles on the road; they represent a broader commitment to sustainable, people-centric urban living. By prioritizing pedestrians, cyclists, and public transit, cities can create environments that prioritize the well-being of residents and contribute to a healthier, more resilient future.""",

    """Car-free zones, when strategically implemented, can serve as models for sustainable urban living. These areas allow for experimentation with innovative transportation solutions, fostering a culture of adaptation, learning, and continuous improvement in the quest for creating truly livable, car-free cities.""",

    """Car-free cities reimagine transportation as a public good rather than a private convenience. By placing emphasis on accessible, efficient public transit and creating pedestrian-friendly spaces, these cities challenge the traditional notion of urban mobility and set the stage for a more equitable and sustainable future.""",

    """The success of car-free cities hinges on the seamless integration of various modes of transportation. Efficient public transit, cycling infrastructure, and pedestrian-friendly designs must coalesce to create a comprehensive and accessible network that caters to the diverse needs of urban residents.""",

    """Car-free cities offer a tangible solution to the dual challenges of traffic congestion and environmental degradation. By fostering a culture of sustainable mobility and prioritizing alternative modes of transportation, cities can create cleaner, more resilient urban environments that benefit both current and future generations.""",

    """The creation of car-free cities requires a holistic approach to urban planning that goes beyond transportation infrastructure. It involves reimagining public spaces, encouraging sustainable living practices, and fostering a sense of community that transcends the traditional boundaries imposed by vehicular traffic.""",

    """Car-free cities challenge the status quo by prioritizing the well-being of residents over the convenience of private vehicles. This paradigm shift requires a concerted effort from urban planners, policymakers, and the community to reimagine urban spaces, embrace sustainable alternatives, and collectively work towards creating more livable, resilient cities.""",
]
len(chatgpt_essays_3)

44

In [16]:
essays_electoral_college = [
    """The efficacy of the Electoral College in the United States has been a subject of debate since its inception. Proponents argue that it ensures a fair representation of all states in the presidential election, preventing populous states from dominating the outcome. However, critics contend that it can lead to situations where the candidate with fewer popular votes wins the election, raising questions about the system's democratic legitimacy.""",

    """The Electoral College was devised as a compromise to balance the interests of smaller and larger states during the formation of the U.S. Constitution. While intended to prevent the dominance of highly populated states, it has faced criticism for not accurately reflecting the popular vote. This raises concerns about the fairness and representativeness of the current electoral system.""",

    """The effectiveness of the Electoral College is a contentious issue in American politics. Supporters argue that it promotes stability and prevents candidates from focusing solely on densely populated urban areas, ensuring that the concerns of a diverse range of states are considered. However, detractors point to instances where the winner of the popular vote did not secure the presidency, prompting calls for electoral reform.""",

    """The question of whether the Electoral College works is complex and multifaceted. On one hand, it provides a mechanism to prevent the marginalization of smaller states in the presidential election process. On the other hand, it has been criticized for deviating from the principle of one person, one vote, as seen in cases where a candidate wins the electoral vote but loses the popular vote. This tension fuels the ongoing debate about the system's effectiveness.""",

    """The Electoral College has been a fixture of American elections, but its efficacy is a matter of ongoing discussion. While it is designed to balance the influence of states with varying populations, the winner-takes-all approach in most states can lead to a distortion of the popular vote. This has led to calls for reform or even the adoption of alternative methods to ensure a more direct and representative connection between citizens and their choice for president.""",

    """Proponents of the Electoral College argue that it prevents candidates from focusing solely on high-population urban areas, ensuring that the concerns of citizens across diverse states are taken into account. However, the winner-takes-all system in most states has faced criticism for not accurately reflecting the popular vote, raising questions about the fairness and legitimacy of the current electoral process.""",

    """The question of whether the Electoral College effectively serves its intended purpose is a divisive one. While it aims to balance the influence of states with different population sizes, it has faced criticism for producing outcomes where the candidate with fewer popular votes secures the presidency. This discrepancy has fueled ongoing debates about the need for electoral reform to better align with democratic principles.""",

    """The Electoral College is a unique feature of the U.S. presidential election system, designed to strike a balance between the interests of large and small states. However, its effectiveness has been scrutinized, particularly when the electoral outcome diverges from the popular vote. This has led to discussions about potential reforms to ensure a more accurate representation of the will of the people in electing the nation's leader.""",

    """The debate over the effectiveness of the Electoral College is deeply rooted in the tension between state and individual representation. While the system aims to prevent the dominance of densely populated states, critics argue that it can lead to situations where the preferences of the majority are not reflected in the electoral outcome. The ongoing discourse raises questions about the adaptability and fairness of the current electoral framework.""",

    """The functionality of the Electoral College has been a topic of scrutiny in American politics. While proponents argue that it protects the interests of smaller states and ensures a geographically diverse representation, critics point to instances where it has deviated from the popular vote. This prompts discussions about the need for reform to align the electoral process more closely with democratic principles.""",

    """The question of whether the Electoral College works as intended is pivotal in discussions about the fairness of the U.S. presidential election system. While it was established to prevent the domination of highly populated states, its winner-takes-all approach in most states has been criticized for distorting the popular vote. This ongoing debate underscores the need for a comprehensive examination of the system's strengths and weaknesses.""",
]
essays_electoral_college_2 = [
    """The Electoral College, designed to balance the interests of states, has been a cornerstone of American elections. While it aims to prevent the marginalization of smaller states, critics argue that it can lead to scenarios where the popular vote does not align with the electoral outcome. This tension prompts ongoing discussions about the efficacy of the Electoral College and the need for potential reforms.""",

    """The Electoral College, a mechanism devised to safeguard the interests of states with varying populations, has been a subject of scrutiny. Despite its original intent, instances where candidates win the electoral vote but lose the popular vote have sparked debates about the system's fairness and representation. This calls into question whether the Electoral College effectively serves the democratic principles of the United States.""",

    """The effectiveness of the Electoral College in ensuring a fair and representative presidential election remains a contentious issue. Advocates argue that it prevents candidates from focusing solely on populous states, while critics point to its potential to distort the popular vote. As the nation grapples with questions about the democratic legitimacy of its electoral process, discussions on electoral reform gain momentum.""",

    """The Electoral College was conceived to strike a balance between the needs of large and small states, yet its functionality has been a subject of ongoing debate. While it offers protection against the dominance of densely populated states, concerns persist regarding its deviation from the popular vote. This prompts a critical examination of the Electoral College and discussions about potential reforms to enhance its democratic alignment.""",

    """As a unique feature of the U.S. electoral system, the Electoral College has been both praised for its balancing act and criticized for potential distortions of the popular vote. The tension between state and individual representation raises questions about the system's overall efficacy. In the quest for a fair and representative election process, the debate on the Electoral College's functionality continues to evolve.""",

    """The role and effectiveness of the Electoral College in the U.S. electoral process are central to discussions about the nation's democratic principles. While it aims to prevent the disproportionate influence of highly populated states, critics argue that it can lead to situations where the majority's will is not accurately reflected. This ongoing discourse prompts a reevaluation of the Electoral College's place in modern democracy.""",

    """Debates surrounding the Electoral College revolve around its ability to balance state interests and ensure a fair representation of all citizens. While it may prevent the dominance of populous states, concerns persist regarding its potential to diverge from the popular vote. The question of whether the Electoral College works becomes a focal point in discussions about electoral integrity and democratic values.""",

    """The Electoral College, a mechanism embedded in the U.S. Constitution, has been integral to the nation's electoral process. Despite its historical significance, questions persist about its efficacy in the modern context. The tension between state and individual representation prompts ongoing discussions about the need for reforms to align the electoral system more closely with democratic ideals.""",

    """The Electoral College, conceived to address the interests of states with varying populations, has been a defining element of American elections. While it aims to prevent the marginalization of smaller states, concerns about its potential to override the popular vote fuel discussions about its functionality and democratic legitimacy. The ongoing debate underscores the need for a nuanced examination of the Electoral College's role in shaping the nation's leadership.""",

    """The question of whether the Electoral College effectively serves the democratic principles of the United States is at the heart of electoral discussions. While it was designed to prevent the dominance of highly populated states, its winner-takes-all approach has faced criticism for distorting the popular vote. This prompts a reexamination of the Electoral College's role in the nation's electoral landscape.""",

    """The Electoral College, as a product of historical compromise, has shaped the U.S. presidential election process. While it prevents the domination of populous states, debates persist about its potential to produce outcomes inconsistent with the popular vote. This ongoing discussion prompts reflections on the effectiveness of the Electoral College in meeting the democratic expectations of the American people.""",

    """The Electoral College, conceived as a mechanism to prevent the dominance of densely populated states, has been both praised and criticized. While it aims to balance state interests, concerns persist about its potential to undermine the popular vote. As the nation grapples with questions about electoral fairness, discussions on potential reforms gain traction to align the system more closely with democratic ideals.""",

    """The functionality of the Electoral College, deeply ingrained in the U.S. electoral system, has been a subject of scrutiny. While it provides a safeguard against the undue influence of populous states, it has faced criticism for producing outcomes incongruent with the popular vote. The ongoing debate prompts a critical examination of the Electoral College's role in shaping the democratic principles of the nation.""",

    """The Electoral College, a cornerstone of the U.S. electoral process, has been a subject of debate regarding its efficacy and democratic alignment. While it aims to prevent the marginalization of smaller states, instances where it has diverged from the popular vote raise questions about its functionality. The ongoing discourse prompts considerations for potential reforms to ensure a more representative electoral system.""",

    """The effectiveness of the Electoral College, rooted in the U.S. Constitution, is a subject of ongoing discourse. While it offers protection against the dominance of highly populated states, concerns about its potential to override the popular vote persist. The question of whether the Electoral College works becomes a focal point in discussions about the fairness and representativeness of the U.S. electoral process.""",

    """The Electoral College, designed as a compromise to balance state interests, has been a feature of U.S. elections for centuries. While it aims to prevent the domination of populous states, debates about its effectiveness persist. The tension between state and individual representation prompts discussions about potential reforms to align the electoral system more closely with democratic principles.""",

    """The role of the Electoral College in the U.S. electoral process has been a subject of ongoing debate. While it provides a safeguard against the dominance of densely populated states, concerns persist regarding its potential to deviate from the popular vote. This prompts a reevaluation of the Electoral College's role in ensuring a fair and representative presidential election.""",

    """The Electoral College, as a unique feature of the U.S. electoral system, has faced scrutiny regarding its democratic alignment. While it seeks to balance state interests, criticisms regarding its potential to distort the popular vote persist. The ongoing discourse prompts considerations for potential reforms to enhance the fairness and representativeness of the nation's electoral process.""",

    """The question of whether the Electoral College effectively serves its intended purpose remains a central point of discussion in American politics. While it aims to prevent the undue influence of populous states, concerns about its potential to override the popular vote raise questions about its functionality. The ongoing debate prompts reflections on the democratic principles underpinning the U.S. electoral system.""",

    """The Electoral College, conceived as a mechanism to prevent the dominance of densely populated states, has played a crucial role in shaping the U.S. presidential election process. While it aims to balance state interests, its potential to produce outcomes inconsistent with the popular vote has fueled discussions about its efficacy and democratic legitimacy. The ongoing debate prompts reflections on the evolving nature of the American electoral landscape.""",

    """The functionality of the Electoral College, enshrined in the U.S. Constitution, is a topic of ongoing debate and reflection. While it prevents the marginalization of smaller states, concerns persist about its potential to override the popular vote. The question of whether the Electoral College works as intended becomes a focal point in discussions about the democratic values of the U.S. electoral system.""",
]
essays_electoral_college_3 = [
    """The Electoral College, a fundamental aspect of the U.S. electoral system, has faced persistent scrutiny regarding its effectiveness. While it aims to prevent the dominance of densely populated states, debates about its potential to distort the popular vote fuel discussions on the need for electoral reform. This ongoing discourse prompts a critical examination of the Electoral College's role in shaping the nation's democratic principles.""",

    """The question of whether the Electoral College effectively serves its intended purpose remains at the forefront of electoral discussions in the United States. While designed to prevent the marginalization of smaller states, criticisms regarding its potential to override the popular vote prompt reflections on the democratic values embedded in the U.S. electoral system.""",

    """The Electoral College, as a product of historical compromise, has been both a stabilizing force and a source of controversy. While it offers protection against the undue influence of populous states, concerns about its potential to deviate from the popular vote have fueled discussions about its efficacy and democratic alignment. The ongoing debate prompts considerations for potential reforms to enhance the fairness and representativeness of the U.S. electoral process.""",

    """The Electoral College, entrenched in the U.S. Constitution, has been a central element of American elections. While it aims to balance the influence of states with varying populations, questions persist about its functionality and alignment with democratic principles. The tension between state and individual representation prompts ongoing discussions about the need for potential reforms to ensure a more representative electoral system.""",

    """The efficacy of the Electoral College in achieving its intended goals is a matter of ongoing debate. While it seeks to prevent the dominance of densely populated states, instances where it has diverged from the popular vote raise questions about its democratic legitimacy. The ongoing discourse prompts reflections on the evolving nature of the U.S. electoral landscape and the potential for reforms to enhance its fairness.""",

    """The Electoral College, with its roots in the founding principles of the United States, remains a subject of scrutiny in modern politics. While it offers a mechanism to prevent the marginalization of smaller states, criticisms about its potential to distort the popular vote persist. This ongoing debate prompts reflections on the democratic ideals underpinning the nation's electoral system and considerations for potential reforms.""",

    """The role and functionality of the Electoral College in the U.S. electoral system have been subjects of persistent debate. While it aims to prevent the dominance of densely populated states, concerns about its potential to override the popular vote prompt ongoing discussions about its democratic alignment. The evolving nature of these discussions underscores the need for critical examination and potential reforms.""",

    """The Electoral College, conceived as a compromise to address state interests, has been a fixture of American elections. While it aims to balance the influence of states with varying populations, debates persist regarding its potential to produce outcomes inconsistent with the popular vote. The ongoing discourse prompts reflections on the historical significance and potential reforms of the Electoral College to better align with democratic principles.""",

    """The question of whether the Electoral College effectively upholds democratic principles remains a central point of discussion. While designed to prevent the undue influence of populous states, criticisms about its potential to deviate from the popular vote fuel ongoing debates. The evolving nature of these discussions prompts considerations for potential reforms to enhance the fairness and representativeness of the U.S. electoral process.""",

    """The Electoral College, embedded in the U.S. electoral system, has been both praised for its historical role and criticized for potential shortcomings. While it seeks to balance the influence of states with different populations, concerns persist about its potential to distort the popular vote. This ongoing debate prompts reflections on the democratic ideals that underpin the nation's electoral process and considerations for potential reforms.""",

    """The role and efficacy of the Electoral College in American elections are subjects of ongoing debate. While it aims to prevent the marginalization of smaller states, concerns about its potential to override the popular vote raise questions about its democratic legitimacy. The ongoing discourse prompts reflections on the historical significance and potential reforms of the Electoral College to better align with contemporary democratic principles.""",

    """The Electoral College, as a unique feature of the U.S. electoral system, has been a subject of scrutiny regarding its functionality. While it provides a safeguard against the dominance of densely populated states, concerns about its potential to distort the popular vote persist. This ongoing debate prompts reflections on the historical significance and potential reforms of the Electoral College to better align with contemporary democratic principles.""",

    """The question of whether the Electoral College effectively serves its intended purpose remains a central point of discussion in American politics. While designed to prevent the undue influence of populous states, criticisms about its potential to deviate from the popular vote fuel ongoing debates. The evolving nature of these discussions prompts considerations for potential reforms to enhance the fairness and representativeness of the U.S. electoral process.""",

    """The functionality of the Electoral College, rooted in the U.S. Constitution, is a topic of ongoing debate and reflection. While it prevents the marginalization of smaller states, concerns persist about its potential to override the popular vote. The question of whether the Electoral College works as intended becomes a focal point in discussions about the democratic values of the U.S. electoral system.""",

    """The Electoral College, conceived as a mechanism to prevent the dominance of densely populated states, has played a crucial role in shaping the U.S. presidential election process. While it aims to balance state interests, its potential to produce outcomes inconsistent with the popular vote has fueled discussions about its efficacy and democratic legitimacy. The ongoing debate prompts reflections on the evolving nature of the American electoral landscape.""",

    """The functionality of the Electoral College, entrenched in the U.S. Constitution, remains a topic of ongoing debate. While it provides a safeguard against the undue influence of populous states, concerns about its potential to distort the popular vote persist. This ongoing debate prompts reflections on the historical significance and potential reforms of the Electoral College to better align with contemporary democratic principles.""",

    """The question of whether the Electoral College effectively serves its intended purpose remains at the forefront of electoral discussions in the United States. While designed to prevent the marginalization of smaller states, criticisms regarding its potential to override the popular vote prompt reflections on the democratic values embedded in the U.S. electoral system.""",

    """The role and functionality of the Electoral College in the U.S. electoral system have been subjects of persistent debate. While it aims to prevent the dominance of densely populated states, concerns about its potential to override the popular vote prompt ongoing discussions about its democratic alignment. The evolving nature of these discussions underscores the need for critical examination and potential reforms.""",

    """The Electoral College, as a unique feature of the U.S. electoral system, has been both praised for its historical role and criticized for potential shortcomings. While it seeks to balance the influence of states with different populations, concerns persist about its potential to distort the popular vote. This ongoing debate prompts reflections on the democratic ideals that underpin the nation's electoral process and considerations for potential reforms.""",

    """The role and efficacy of the Electoral College in American elections are subjects of ongoing debate. While it aims to prevent the marginalization of smaller states, concerns about its potential to override the popular vote raise questions about its democratic legitimacy. The ongoing discourse prompts reflections on the historical significance and potential reforms of the Electoral College to better align with contemporary democratic principles.""",

    """The Electoral College, embedded in the U.S. electoral system, has been both praised for its historical role and criticized for potential shortcomings. While it seeks to balance the influence of states with different populations, concerns persist about its potential to distort the popular vote. This ongoing debate prompts reflections on the democratic ideals that underpin the nation's electoral process and considerations for potential reforms.""",

    """The question of whether the Electoral College effectively upholds democratic principles remains a central point of discussion. While designed to prevent the undue influence of populous states, criticisms about its potential to deviate from the popular vote fuel ongoing debates. The evolving nature of these discussions prompts considerations for potential reforms to enhance the fairness and representativeness of the U.S. electoral process.""",

    """The functionality of the Electoral College, rooted in the U.S. Constitution, is a topic of ongoing debate and reflection. While it prevents the marginalization of smaller states, concerns persist about its potential to override the popular vote. The question of whether the Electoral College works as intended becomes a focal point in discussions about the democratic values of the U.S. electoral system.""",

    """The Electoral College, conceived as a mechanism to prevent the dominance of densely populated states, has played a crucial role in shaping the U.S. presidential election process. While it aims to balance state interests, its potential to produce outcomes inconsistent with the popular vote has fueled discussions about its efficacy and democratic legitimacy. The ongoing debate prompts reflections on the evolving nature of the American electoral landscape.""",

    """The functionality of the Electoral College, entrenched in the U.S. Constitution, remains a topic of ongoing debate. While it provides a safeguard against the undue influence of populous states, concerns about its potential to distort the popular vote persist. This ongoing debate prompts reflections on the historical significance and potential reforms of the Electoral College to better align with contemporary democratic principles.""",
]

In [17]:
#total generated essays using ChatGPT
ai_generated = chatgpt_essays + chatgpt_essays_3 + chatgpt_esssays_2 + essays_electoral_college + essays_electoral_college_2 + essays_electoral_college_3
len(ai_generated)

154

In [18]:
ai_generated = ai_generated + train_df[train_df['generated']==1]['text'].tolist()
len(ai_generated)

157

In [19]:
#Selecting human writtenn from each class randomly
human_written_1 = train_df[(train_df['generated'] == 0) & (train_df['prompt_id'] == 1)]['text'].sample(80, random_state=42).tolist()
human_written_2 = train_df[(train_df['generated'] == 0) & (train_df['prompt_id'] == 0)]['text'].sample(80, random_state=42).tolist()
human_written = human_written_1 + human_written_2
len(human_written)

160

In [20]:
import numpy as np
human_labels = [0]*len(human_written)
ai_labels = [1]*len(ai_generated)
y = human_labels + ai_labels
print(y)
len(y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


317

In [21]:
texts = list(human_written) + list(ai_generated)

len(texts)

317

In [22]:
#Final Data
df = pd.DataFrame({'texts': texts, 'label': y})

# Display the DataFrame
print(df)


                                                 texts  label
0    Dear State Senator, we should not keep the Ele...      0
1    Dear State Senator, The Electoral College is a...      0
2    The Electoral Collage has been around for a lo...      0
3    Dear United States Senator, Voting in the Unit...      0
4    Dear State Senator, I am writing you a letter ...      0
..                                                 ...    ...
312  The Electoral College, conceived as a mechanis...      1
313  The functionality of the Electoral College, en...      1
314  This essay will analyze, discuss and prove one...      1
315  I strongly believe that the Electoral College ...      1
316  Limiting car use causes pollution, increases c...      1

[317 rows x 2 columns]


In [23]:
import string

def preprocess_text(text):
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text_no_punct = text.translate(translator)

    # Convert to lowercase
    text_lower = text_no_punct.lower()

    return text_lower
df['texts'] = df['texts'].apply(preprocess_text)
df.head()

,texts,label
0,dear state senator we should not keep the elec...,0
1,dear state senator the electoral college is a ...,0
2,the electoral collage has been around for a lo...,0
3,dear united states senator voting in the unite...,0
4,dear state senator i am writing you a letter i...,0


### Naive Bayes

In [24]:
#Build Vocabularies
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [25]:
def build_vocabulary(texts):
    """
    Build a vocabulary from a list of texts.
    Returns a set of unique words.
    """
    vocabulary = set()

    for text in texts:
        # Tokenize the text
        words = word_tokenize(text.lower())

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in words if word not in stop_words]

        # Add unique words to the vocabulary
        vocabulary.update(filtered_words)

    return vocabulary

def get_top_words(texts, top_n=10):
    """
    Get the top N words based on their frequency in a list of texts.
    Returns a list of tuples (word, frequency).
    """
    # Tokenize and preprocess each text
    tokenized_texts = [word_tokenize(text.lower()) for text in texts]

    # Flatten the list of lists
    all_words = [word for sublist in tokenized_texts for word in sublist]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in all_words if word not in stop_words]

    # Count word frequencies
    word_freq = Counter(filtered_words)

    # Get the top N words
    top_words = word_freq.most_common(top_n)

    return top_words

In [26]:
human_vocab = build_vocabulary(df[df['label']==0]['texts'].tolist())
llm_vocab = build_vocabulary(df[df['label']==1]['texts'].tolist())

In [27]:
list(human_vocab)[:50]

['ares',
 'posner',
 'influencing',
 'experienced',
 'promoted',
 'moeny',
 'par32people',
 'accident',
 'focuses',
 'bu',
 'individuals',
 'extora',
 'dole',
 'safe',
 'employees',
 'coming',
 'storngly',
 'reuters',
 'un',
 'collectoion',
 'ur',
 'suvi',
 'americasource',
 'arguments',
 'stable',
 'inhabitants',
 'effie',
 'concept',
 'prefer',
 'asks',
 'half',
 'improves',
 'recent',
 'completley',
 'destroyed',
 'computers',
 'within',
 'onegatively',
 'te',
 'converging',
 'pull',
 'worrying',
 'supporters',
 'necklace',
 'commerce',
 'single',
 'electricity',
 'unites',
 'p18',
 'efficieoncy']

In [28]:
list(llm_vocab)[0:50]

['navigate',
 'change',
 'unique',
 'accommodates',
 'task',
 'areas',
 'environmentally',
 'equity',
 'fundamental',
 'compromising',
 'places',
 'gain',
 'successfully',
 'local',
 'vibrant',
 'progressive',
 'representativeness',
 'citizen',
 'momentum',
 'electric',
 'starting',
 'ability',
 'shared',
 'key',
 'improved',
 'accident',
 'lead',
 'densely',
 'transforming',
 'onto',
 'redefining',
 'still',
 'increased',
 'individuals',
 'global',
 'controversy',
 'drastic',
 'intended',
 'implemented',
 'strike',
 'vehicular',
 'forefront',
 'land',
 'becomes',
 'helps',
 'carfree',
 'bound',
 'belonging',
 'promise',
 'pushed']

In [29]:
top_20_humanwords = get_top_words(df[df['label']==0]['texts'].tolist(),20)
top_20_humanwords

[('electoral', 918),
 ('people', 842),
 ('vote', 755),
 ('college', 734),
 ('cars', 704),
 ('car', 635),
 ('states', 524),
 ('would', 461),
 ('president', 426),
 ('electors', 351),
 ('votes', 350),
 ('popular', 289),
 ('one', 282),
 ('state', 267),
 ('voters', 262),
 ('many', 257),
 ('even', 257),
 ('get', 256),
 ('election', 245),
 ('use', 241)]

In [30]:
top_20_llmwords = get_top_words(df[df['label']==1]['texts'].tolist(),20)
top_20_llmwords

[('electoral', 150),
 ('carfree', 113),
 ('cities', 98),
 ('college', 81),
 ('states', 78),
 ('urban', 76),
 ('potential', 66),
 ('vote', 64),
 ('popular', 56),
 ('transportation', 54),
 ('ongoing', 47),
 ('democratic', 46),
 ('us', 41),
 ('initiatives', 37),
 ('public', 34),
 ('residents', 34),
 ('prompts', 33),
 ('debate', 32),
 ('discussions', 32),
 ('traffic', 29)]

In [31]:
#Splitting the data
from sklearn.model_selection import train_test_split
train_df, dev_df = train_test_split(df, test_size=0.2, random_state=42)

In [32]:
documents = train_df['texts']
labels = train_df['label']

In [33]:
import re
from collections import defaultdict
from math import log, exp

class NaiveBayesClassifier:
    def __init__(self):
        self.class_probs = defaultdict(float)
        self.word_probs = defaultdict(lambda: defaultdict(float))
        self.classes = set()

    def train(self, documents, labels, alpha):
        # Count occurrences of each class
        class_counts = defaultdict(int)
        for label in labels:
            class_counts[label] += 1
            self.classes.add(label)

        total_documents = len(labels)

        # Calculate class probabilities
        for label, count in class_counts.items():
            self.class_probs[label] = count / total_documents

        # Count word occurrences in each class with Laplace smoothing
        word_counts = defaultdict(lambda: defaultdict(int))

        for doc, label in zip(documents, labels):
            words = re.findall(r'\b\w+\b', doc.lower())
            for word in words:
                word_counts[label][word] += 1

        # Calculate word probabilities for each class with Laplace smoothing
        for label in self.classes:
            total_words_in_class = sum(word_counts[label].values())
            total_unique_words = len(set(word_counts[label].keys()))

            for word, count in word_counts[label].items():
                self.word_probs[label][word] = (count + alpha) / (total_words_in_class + alpha * total_unique_words)
                
    def get_top_words_per_class(self, top_n=10):
        top_words_per_class = defaultdict(list)

        for label in self.classes:
            word_probabilities = self.word_probs[label]
            top_words = sorted(word_probabilities, key=word_probabilities.get, reverse=True)[:top_n]
            top_words_per_class[label] = top_words

        return top_words_per_class

    def predict(self, documents):
        # Assuming 'documents' is a list of documents
        probabilities_list = []

        for document in documents:
            words = re.findall(r'\b\w+\b', str(document).lower())
            scores = defaultdict(float)

            for label in self.classes:
                scores[label] = log(self.class_probs[label])

                for word in words:
                    scores[label] += log(self.word_probs[label].get(word, 1e-10))

            exp_scores = {label: exp(score) for label, score in scores.items()}
            sum_exp_scores = sum(exp_scores.values())

            if sum_exp_scores == 0:
                probabilities = {label: 1 / len(self.classes) for label in self.classes}
            else:
                probabilities = {label: exp_score / sum_exp_scores for label, exp_score in exp_scores.items()}

            probabilities_list.append(probabilities)

        return probabilities_list




# Create and train the classifier
alpha = [0.0, 0.1, 0.5, 1.0, 5.0]


dev_data_documents = dev_df['texts']
dev_data_labels = dev_df['label']
best_accuracy = 0
best_alpha = 0
for alpha_val in alpha:
    result = []
    classifier = NaiveBayesClassifier()
    classifier.train(documents, labels, alpha_val)
    train_prob_list = classifier.predict(dev_data_documents)
    for i in range(len(train_prob_list)):
        if(train_prob_list[i][0] >= train_prob_list[i][1]):
            result.append(0)
        else:
            result.append(1)
    count = 0
    for i in range(len(result)):
        if(result[i] == dev_data_labels.iloc[i]):
            count = count + 1

    accuracy = count / len(result)
    
    if(accuracy > best_accuracy):
        best_accuracy = accuracy
        best_alpha = alpha_val
    
    print("For alpha value:", alpha_val, "Accuracy:", accuracy)
    
print("For Best Alpha value:", best_alpha, "Accuracy:", best_accuracy)

best_classifier = NaiveBayesClassifier()
best_classifier.train(documents, labels, best_alpha)
top_words = best_classifier.get_top_words_per_class()
for label, words in top_words.items():
    print(f"Top words for class '{label}': {', '.join(words)}")

For alpha value: 0.0 Accuracy: 1.0
For alpha value: 0.1 Accuracy: 1.0
For alpha value: 0.5 Accuracy: 1.0
For alpha value: 1.0 Accuracy: 1.0
For alpha value: 5.0 Accuracy: 1.0
For Best Alpha value: 0.0 Accuracy: 1.0
Top words for class '0': the, to, of, a, in, and, is, that, for, it
Top words for class '1': the, of, and, to, a, electoral, for, in, carfree, its


In [34]:
dev_data_documents = dev_df['texts']
dev_data_labels = dev_df['label']
train_prob_list = best_classifier.predict(dev_data_documents)

In [35]:
test_df['text'] = test_df['text'].apply(preprocess_text)

In [36]:
test_documents = test_df['text']
probabilities_list = classifier.predict(test_documents)

print(probabilities_list)

[{0: 0.5019762845849818, 1: 0.4980237154150182}, {0: 0.5019762845849818, 1: 0.4980237154150182}, {0: 0.5019762845849818, 1: 0.4980237154150182}]


In [37]:
proba = []
for i in probabilities_list:
    proba.append(i[1])

predicted = np.array(proba)

In [38]:
output= pd.DataFrame({'id':test_df["id"],'generated':predicted})
output.to_csv('submission.csv', index=False)